In [7]:
from torch_geometric.data import HeteroData
import torch
import pandas as pd
import numpy as np

In [11]:
users_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/users.csv")
products_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/products.csv")
user_product_edge_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/user_product_edges.csv")
also_bought_edges_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/also_bought_edges.csv")
also_viewed_edges_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/also_viewed_edges.csv")
bought_together_edges_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/bought_together_edges.csv")

In [43]:
products_pd = pd.read_csv("C:/Users/josep/amazon_dataset_latest/all_products.csv")

In [19]:
users_pd.head()

,overall,reviewerID,asin,date,helpfulness_ratio
0,5,A2XVJBSRI3SWDI,0000031887,2013-11-04,0.0
1,2,A2R3K1KX09QBYP,0000031887,2013-02-17,1.0
2,1,A19PBP93OF896,0000031887,2013-03-21,0.0
3,4,A1P0IHU93EF9ZK,0000031887,2014-01-23,0.0
4,5,A2G5TCU2WDFZ65,0000031887,2013-01-19,0.0


In [44]:
products_pd.head()

,asin,title,price,imUrl,brand,description,categories,category,also_viewed,also_bought,bought_together
0,0000037214,Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,6.99,http://ecx.images-amazon.com/images/I/31mCncNu...,Big Dreams,NaN,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",clothing_shoes_jewellery,['B00JO8II76'],NaN,NaN
1,0000031887,Ballet Dress-Up Fairy Tutu,6.79,http://ecx.images-amazon.com/images/I/314qZjYe...,Boutique Cutie,This adorable basic ballerina tutu is perfect ...,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",clothing_shoes_jewellery,"['B00538F5OK', 'B003AVKOP2', 'B008F0SU0Y', 'B0...","['B003AVKOP2', 'B00D103F8U', 'B008F0SU0Y', 'B0...",[]
2,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,64.98,http://ecx.images-amazon.com/images/I/413tGhqo...,NaN,Elegance par excellence. Hand-crafted of the f...,"[['Clothing, Shoes & Jewelry', 'Novelty, Costu...",clothing_shoes_jewellery,"['B0006JCGUM', 'B000BMTCK6', 'B00194Q262', 'B0...","['B000BMTCK6', 'B0006JCGUM', 'B00194Q262', 'B0...","['B000BMTCK6', 'B0006JCGUM']"
3,0456844570,RiZ Women's Beautify Crafted &frac12; Rimmed F...,NaN,http://ecx.images-amazon.com/images/I/31QZTHxv...,NaN,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",clothing_shoes_jewellery,"['B004UOPY80', 'B00G2LY6OC', 'B00877BFFM', 'B0...",NaN,NaN
4,0456808574,Lantin White Visor Wrap Around Ski Style Aviat...,NaN,http://ecx.images-amazon.com/images/I/31UsrgT5...,NaN,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",clothing_shoes_jewellery,"['B005WF42SY', 'B003T1CFF0', 'B006AZCMUA', 'B0...",NaN,NaN


In [28]:
user_product_edge_pd.head()

,overall,reviewerID,asin,date,helpfulness_ratio
0,5,A2XVJBSRI3SWDI,0000031887,2013-11-04,0.0
1,2,A2R3K1KX09QBYP,0000031887,2013-02-17,1.0
2,1,A19PBP93OF896,0000031887,2013-03-21,0.0
3,4,A1P0IHU93EF9ZK,0000031887,2014-01-23,0.0
4,5,A2G5TCU2WDFZ65,0000031887,2013-01-19,0.0


In [15]:
also_bought_edges_pd.head()

,asin,also_bought
0,0000031887,B003AVKOP2
1,0000031887,B00D103F8U
2,0000031887,B008F0SU0Y
3,0000031887,B003AVEU6G
4,0000031887,B002C3Y6WG


In [17]:
also_viewed_edges_pd.head()

,asin,also_viewed
0,0000037214,B00JO8II76
1,0000031887,B00538F5OK
2,0000031887,B003AVKOP2
3,0000031887,B008F0SU0Y
4,0000031887,B002GZPTRQ


In [18]:
bought_together_edges_pd.head()

,asin,bought_together
0,0123456479,B000BMTCK6
1,0123456479,B0006JCGUM
2,1608322130,B003USWPHA
3,245600258X,B004LK9GZ0
4,3293015344,B007VE0OI6


In [55]:
class AsinIdMap:
    def __init__(self, products_pd):
        self.products_pd = products_pd
        self.asin_to_idx = {}
        self.product_idx_to_asin = {}
        self._build()
    
    def _build(self):
        for idx, row in self.products_pd.iterrows():
            self.asin_to_idx[row['asin']] = idx
            self.product_idx_to_asin[idx] = row['asin']
    
    def get_idx(self, asin):
        return self.asin_to_idx[asin]
    
    def get_asin(self, product_idx):
        return self.product_idx_to_asin[product_idx]
    
    def get_count(self):
        return len(self.asin_to_idx)
    
class UserIdMap:
    def __init__(self, users_pd, offset=0):
        self.users_pd = users_pd
        self.user_id_to_idx = {}
        self.user_idx_to_id = {}
        self.offset = offset
        self._build()
    
    def _build(self):
        for idx, row in self.users_pd.iterrows():
            self.user_id_to_idx[row['reviewerID']] = self.offset + idx
            self.user_idx_to_id[idx] = row['reviewerID']
    
    def get_idx(self, user_id):
        return self.user_id_to_idx[user_id]
    
    def get_reviewer_id(self, user_idx):
        return self.user_idx_to_id[user_idx]

def build_product_to_product_edge_index(edges_pd, asin_id_map, right_key):
    product_to_product_edge_index = []
    errors = []
    for idx, row in edges_pd.iterrows():
        try:
            asin1 = row['asin']
            product1_idx = asin_id_map.get_idx(asin1)
        except:
            errors.append(f"Cannot map {asin1} to product idx")
            continue
        
        try:
            asin2 = row[right_key]
            product2_idx = asin_id_map.get_idx(asin2)
        except:
            errors.append(f"Cannot map {asin2} to product idx")
            continue
            
        product_to_product_edge_index.append([product1_idx, product2_idx])
        
    return torch.tensor(product_to_product_edge_index).t().contiguous(), errors
    
def build_user_product_edge_index(user_product_edge_pd, asin_id_map, user_id_map):
    user_product_edge_index = []
    errors = []
    for idx, row in user_product_edge_pd.iterrows():
        try:
            user_idx = user_id_map.get_idx(row['reviewerID'])
        except:
            errors.append(f"Cannot map {row['reviewerID'] } to user idx")
            continue
            
        try:
            product_idx = asin_id_map.get_idx(row['asin'])
        except:
            errors.append(f"Cannot map {row['asin'] } to product idx")
            continue
            
        user_product_edge_index.append([user_idx, product_idx])
        
    return torch.tensor(user_product_edge_index).t().contiguous(), errors

In [45]:
asin_id_map = AsinIdMap(products_pd)

In [38]:
user_id_map = UserIdMap(users_pd, offset=asin_id_map.get_count())

In [52]:
user_product_edge_tensor, errors = build_user_product_edge_index(user_product_edge_pd, asin_id_map, user_id_map)

In [ ]:
also_bought_edges_tensor, errors =  build_product_to_product_edge_index(also_bought_edges_pd, asin_id_map, 'also_bought')

In [ ]:
also_viewed_edges_tensor, errors =  build_product_to_product_edge_index(also_viewed_edges_pd, asin_id_map, 'also_viewed')

In [ ]:
bought_together_edges_tensor, errors =  build_product_to_product_edge_index(bought_together_edges_pd, asin_id_map, 'bought_together')

In [3]:
data = HeteroData()

In [6]:
# Dimension for product features
product_feature_dim = 128
num_products = len(products_pd)

num_edges_also_viewed = 1000
num_edges_bought_together = 1000
num_edges_also_bought = 1000

data['product'].x = torch.randn(num_products, product_feature_dim)


data['product', 'also_viewed', 'product'].edge_index = also_viewed_edges_tensor
data['product', 'bought_together', 'product'].edge_index = bought_together_edges_tensor
data['product', 'also_bought', 'product'].edge_index = also_bought_edges_tensor

data

HeteroData(
  product={ x=[100, 128] },
  (product, also_viewed, product)={ edge_index=[2, 1000] },
  (product, bought_together, product)={ edge_index=[2, 1000] },
  (product, also_bought, product)={ edge_index=[2, 1000] }
)